In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# Reading the data
train_data=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/data/train.csv')
test_data=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/data/test.csv')
sample_data=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/data/sample_submission.csv')

In [ ]:
#Dropping the id column in sample data
sample_data=sample_data.drop(columns='id',axis=1)
sample_data.shape,test_data.shape

((36183, 1), (36183, 12))

In [ ]:
test_df=pd.concat([test_data,sample_data],axis=1)
test_df.shape,train_data.shape

((36183, 13), (54273, 13))

In [ ]:
# Combining train and test data samples
df=pd.concat([train_data,test_df],axis=0)
df.shape

(90456, 13)

In [ ]:
# Y dataset
Y=df.loc[:,['price']]
Y.head()

,price
0,11000.0
1,8250.0
2,15000.0
3,63500.0
4,7850.0


In [ ]:
# X dataset
X=df.drop(columns=['price'],axis=1)
X.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes


In [ ]:
X.shape,Y.shape

((90456, 12), (90456, 1))

In [ ]:
# Data Transformation
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
cat_features=X.select_dtypes(include='object').columns
num_features=X.select_dtypes(exclude='object').columns
onehot_transform=OneHotEncoder()
num_transform=StandardScaler()
pre_process_train=ColumnTransformer([('OneHotED',onehot_transform,cat_features),('stdscalar',num_transform,num_features)])
X=pre_process_train.fit_transform(X)

In [ ]:
X.shape

(90456, 3460)

In [ ]:
from sklearn.feature_selection import SelectKBest,f_regression
selection=SelectKBest(f_regression,k=50)
X=selection.fit_transform(X,Y)
X.shape

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(90456, 50)

In [ ]:
from sklearn.decomposition import PCA
X=X.toarray()
pca=PCA(n_components=20)
X=pca.fit_transform(X)
X.shape

(90456, 20)

In [ ]:
# Train Test split of data
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.4)

In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((54273, 20), (36183, 20), (54273, 1), (36183, 1))

In [ ]:
# Model building
#!pip install tensorflow[and-cuda]==2.12.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout,ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1, l2

In [ ]:
#Initializing Ann
def model():
  regresser=Sequential()
#Dense fully connected NN ,20 neurons,relu to mitigate vanishing gradient,
  regresser.add(Dense(units=30,activation='relu',input_dim=X_train.shape[1]))
  #Adding the 1st hidden layer
  regresser.add(Dense(units=20,activation='relu',kernel_regularizer=l2(0.01)))
#dropout layer
  regresser.add(Dropout(0.2))

#Adding the 2nd hidden layer
  regresser.add(Dense(units=20, activation='relu', kernel_regularizer=l2(0.01)))

#Adding the 3rd hidden layer
  regresser.add(Dense(units=10,activation='relu'))
  regresser.add(Dropout(0.2))
#Adding the 4th hidden layer

  regresser.add(Dense(units=10,activation='relu'))
#Adding the output layer
  regresser.add(Dense(units=1,activation='linear'))
  #Build and compilie the Ann
# No need to use metrics since loss RMSE is already a metrics
  regresser.compile(optimizer=Adam(learning_rate=0.01),loss='mean_squared_error')
#classifier.compile(optimizer='adam',loss='mean_squared_error')
  return regresser

In [ ]:
#Ensemble model for deep learning
num_models=5
ensemble_model=[model() for i in range(num_models)]

In [ ]:
X_train_dense = X_train.toarray()

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [ ]:
for i in ensemble_model:
    i.fit(X_train, Y_train, epochs=20, batch_size=10, validation_split=0.2)

Epoch 1/20
4342/4342 [==============================] - 35s 7ms/step - loss: 3173251584.0000 - val_loss: 3143635968.0000
Epoch 2/20
4342/4342 [==============================] - 18s 4ms/step - loss: 3109552128.0000 - val_loss: 3158665216.0000
Epoch 3/20
4342/4342 [==============================] - 18s 4ms/step - loss: 3075683584.0000 - val_loss: 3260634112.0000
Epoch 4/20
4342/4342 [==============================] - 15s 3ms/step - loss: 3059768320.0000 - val_loss: 3304544256.0000
Epoch 5/20
4342/4342 [==============================] - 15s 4ms/step - loss: 3042524160.0000 - val_loss: 3226444032.0000
Epoch 6/20
4342/4342 [==============================] - 16s 4ms/step - loss: 3042254592.0000 - val_loss: 3426958848.0000
Epoch 7/20
4342/4342 [==============================] - 16s 4ms/step - loss: 3029930240.0000 - val_loss: 3526201344.0000
Epoch 8/20
4342/4342 [==============================] - 16s 4ms/step - loss: 3021430528.0000 - val_loss: 3264541440.0000
Epoch 9/20
4342/4342 [==========

In [ ]:
bbvvcvcxrzfedeee552444e34r42rewew32222ewe334rssferfghmlfeersyggsghzfjjgaafyjdqetytss  1q345788

In [ ]:
#Making predictions
Y_pred=[i.predict(X_test) for i in ensemble_model]
#Average predictions
import numpy as np
pred=np.mean(Y_pred,axis=0)

1131/1131 [==============================] - 3s 3ms/step


In [ ]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(Y_test,pred, squared=False)
print("Ensemble RMSE:", rmse)

Ensemble RMSE: 54511.824943924825


In [ ]:
# Predicted data to dataframe
pred_data=pd.DataFrame(pred)

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
mse=mean_squared_error(Y_test,pred_data)
rmse=np.sqrt(mse)

In [ ]:
#
rmse

In [ ]:
pred_data.shape,Y_test.shape

((36183, 1), (36183, 1))

In [ ]:
pred_data.insert(0,'id',test_data['id'],True)
pred_data.rename(columns={0:'price'},inplace=True)
pred_data.head()

,id,price
0,54273,23547.843750
1,54274,57301.574219
2,54275,21147.576172
3,54276,46486.625000
4,54277,20564.783203


In [ ]:
pred_data['price']=pred_data['price'].apply(lambda x: format(float(x),".3f"))
pred_data.head()

,id,price
0,54273,23547.844
1,54274,57301.574
2,54275,21147.576
3,54276,46486.625
4,54277,20564.783


In [ ]:
predicted_data=pred_data.to_csv('/content/gdrive/MyDrive/Colab Notebooks/data/prediction.csv',sep=',',index=False)